# Comparing sets in the Rust Standard Library
### David Rackerby

### Hypothesis
Most major programming language's claim that their official implementations of a data structur are "better than anything you could write yourself." Rust's standard containers (`std::collections`) come closer to that ideal than most languages due to their modern implementations of classical data structures. For example, in Rust 1.73, the HashSet uses SIMD for lookups and B-Trees to implement ordered sets. The HashSet–with a time of $O(1)$ amortized for all standard operations–is expected to outperform the BTreeSet, which runs in $O(lg(n))$ time for the same operations. However, it's not officially documented for which input sizes does BTreeSet perform better than the HashMap. Due to implementation reasons, it's expected that for small input sizes the BTreeSet will be faster due to the smaller overhead of comparisons versus hashing. We hypothesize that the range of values for which the B-Tree performs better is between 0 and 10,000 elements.

### Methods
The same data inserted into both sets is pseudo-randomly generated. Insertion times will be measured until the total insertion time for either of the sets surpasses 3 seconds. The version of Rust used is Rust 1.73. The version of Python used is 3.11.2 The version of Matplotlib used is 3.7.1. The pseudo-random number generator used is the ChaCha random number generator. The notebook used to produce this report can be found at https://github.com/rikipls/CSE-431-Honors

```rust
use rand::{Rng, SeedableRng};
use rand_chacha::ChaCha8Rng;

use std::collections::{HashSet, BTreeSet};
use std::error::Error;
use std::time::Instant;
use std::fs::File;
use std::io::Write;

const OUT_DIR: &str = "out";
const SEED: u64 = 431;
const NUM_ELEMENTS: usize = 10_000_000;

fn main() -> Result<(), Box<dyn Error>> {
    let mut rng = ChaCha8Rng::seed_from_u64(SEED);

    // Generate the random data
    let data: Vec<usize> = (0..NUM_ELEMENTS).map(|_| rng.gen_range(0..usize::MAX)).collect();

    // Copies
    let mut hashmap_time: Vec<u128> = vec![0; NUM_ELEMENTS];
    let mut btreemap_time: Vec<u128>  = vec![0; NUM_ELEMENTS];
    let mut total_hashmap_time = 0.0;
    let mut total_btreemap_time = 0.0;

    let mut hash_map= HashSet::<usize>::new();
    let mut btree_map = BTreeSet::<usize>::new();

    let mut num_elements_processed = NUM_ELEMENTS;

    for i in 0..NUM_ELEMENTS {
        let val = data[i];

        // HashMap
        let now_hashmap = Instant::now();
        hash_map.insert(val);
        let hash_elapsed = now_hashmap.elapsed();
        hashmap_time[i] = hash_elapsed.as_nanos();
        total_hashmap_time += hash_elapsed.as_secs_f64();

        // BTreemap
        let now_btreemap = Instant::now();
        btree_map.insert(val);
        let btree_elapsed = now_btreemap.elapsed();
        btreemap_time[i] = btree_elapsed.as_nanos();
        total_btreemap_time += btree_elapsed.as_secs_f64();

        // Stop after total elapsed time exceeds 3 seconds
        if total_btreemap_time >= 3.0 || total_hashmap_time >= 3.0 {
            num_elements_processed = i;
            break;
        }
    }

    let nonzero_hashmap_times_in_ns= &hashmap_time[..num_elements_processed];
    let nonzero_btreemap_times_in_ns= &btreemap_time[..num_elements_processed];

    let hash_times_str: Vec<String> = nonzero_hashmap_times_in_ns.iter().map(|n| format!("{n}")).collect();
    let tree_times_str: Vec<String> = nonzero_btreemap_times_in_ns.iter().map(|n| format!("{n}")).collect();

    let mut f_hash = File::create(format!("{OUT_DIR}/hash.txt"))?;
    let mut f_tree = File::create(format!("{OUT_DIR}/tree.txt"))?;

    writeln!(f_hash, "{}", hash_times_str.join("\n"))?;
    writeln!(f_tree, "{}", tree_times_str.join("\n"))?;

    Ok(())
}
```

### Results
In a given run, the insertion time of the sorted dictionary was typically one order of magnitude higher than that of the unsorted dictionary. As shown above, the sorted dictionary spent much more time overall on insertions than the unsorted dictionary.

### Discussion
As expected, the sorted dictionary took more time than the unsorted dictionary to insert the same elements due to having to maintain the order of keys in its backend. However, what's surprising is that total of 1.4 million elements were inserted before the total insertion time took 3 seconds. One order of magnitude difference is a smaller gap than expected for that many elements, and the SortedDict still performed very well given its asymptotic restriction.

### Conclusions
The unsorted dictionary spent less time on insertions overall. It took 3 seconds to insert roughly 1.4 million elements into the sorted dictionary while it took 0.36 seconds to insert the same number of elements into the unsorted dictionary.